In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    return polynomialCoefficient(dg,polynomial)
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def primesToN(n):  # lists primes p w. 2 <= p <= n
    answer=[]
    for k in [1..n-1]:
        nk=next_prime(k)
        if (nk in answer)==False:
            if nk<n+1:
                answer=answer+[next_prime(k)]
    return answer

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def lastInList(listo):
    ln=len(listo)
    return listo[ln-1]

In [2]:
print primesToN(50)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]


In [2]:
print(j(5,3));print("");print(J(5,3))

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4

x^-1 + 31/72 + 1823/27648*x + 10495/2519424*x^2 + 1778395/18345885696*x^3 + 45767/34828517376*x^4 + 41650330075/3327916660110655488*x^5 + O(x^6)


In [2]:
data=[]
for m in [3..500]:
    if 10 in divisors(m):
        print(m)
    data=data+[[m,j(120,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no6.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500


In [4]:
# above interpolating polynomials look correct, so do the full routine:
import pickle
rfile = open('/Users/barrybrent/8mar21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/10mar21no8.txt','w')
enoughfile=[]
for qpower in [0..120]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=x*s[k][1] 
# times x because, to use polynomialCoefficient, I need a polynomial; not a Laurent series
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower-1,rl]] 
    finish = time.time()
    print([qpower,finish-start])
# -1 because I want to record the coefficients of the qpowers of the original Laurent series
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

[0, 3.476809024810791]
[1, 3.5383589267730713]
[2, 3.4991610050201416]
[3, 3.594343900680542]
[4, 3.532088041305542]
[5, 3.5358738899230957]
[6, 3.5763092041015625]
[7, 3.5106911659240723]
[8, 3.568726062774658]
[9, 3.5482139587402344]
[10, 3.553571939468384]
[11, 3.594465970993042]
[12, 3.520862102508545]
[13, 3.532066822052002]
[14, 3.661604166030884]
[15, 3.5511369705200195]
[16, 3.592189073562622]
[17, 3.567767858505249]
[18, 3.59506893157959]
[19, 3.631237030029297]
[20, 3.5660011768341064]
[21, 3.608276844024658]
[22, 3.559959888458252]
[23, 3.584864854812622]
[24, 3.6208600997924805]
[25, 3.5829501152038574]
[26, 3.615128993988037]
[27, 3.6606009006500244]
[28, 3.621027946472168]
[29, 3.6795859336853027]
[30, 3.7123470306396484]
[31, 3.688579797744751]
[32, 3.6764211654663086]
[33, 3.658989191055298]
[34, 3.6817078590393066]
[35, 3.6548259258270264]
[36, 3.682342767715454]
[37, 3.721776008605957]
[38, 3.702333927154541]
[39, 3.7165682315826416]
[40, 3.744623899459839]
[41, 3.689

In [5]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [0..4]:
    print(s[n][0])
    f=s[n][1]
    print(f);print(f.factor())
    print("---------------------------------------------------------------------------------")

-1
1
1
---------------------------------------------------------------------------------
0
24*x^3 + 32*x
8*(3*x^2 + 4)*x
---------------------------------------------------------------------------------
1
276*x^6 - 32*x^4 - 192*x^2
4*(69*x^4 - 8*x^2 - 48)*x^2
---------------------------------------------------------------------------------
2
2048*x^9 - 237568/27*x^7 + 32768/27*x^5 + 131072/27*x^3
2048/27*(27*x^4 - 8*x^2 - 16)*(x + 2)*(x - 2)*x^3
---------------------------------------------------------------------------------
3
11202*x^12 - 122272*x^10 + 332480*x^8 - 51712*x^6 - 155136*x^4
2*(5601*x^6 - 38732*x^4 + 11312*x^2 + 19392)*(x + 2)*(x - 2)*x^4
---------------------------------------------------------------------------------


In [2]:
import pickle
rsfile = open('/Users/barrybrent/10mar21no8.txt','r')
# SageMath file
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)
rtfile = open('/Users/barrybrent/run20nov20no6','r')
# Mathematica file
L=rtfile.readlines()
rtfile.close()
data=[]
for n in [1..98]:
    f=s[n][1]
    g = D(L[n-1])
    data=data+[expand(D(f-g))]
print[data]

# This shows that the SageMath file and the Mathematica file essentially agree.



[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [18]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if abs(lp-pp)>2:data=data+[index]
print data

[]


In [17]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
no1=[]
no2=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if abs(lp-pp)==2:
        data=data+[index]
        if is_prime(index)==False:no1=no1+[index]
    if is_prime(index)==True:
        if pp-lp!=2:
            no2=no2+[index]
print no1
print no2

[]
[2]


In [3]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
yes=[]
no=[]
twoinpf=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if is_prime(index)==True:
        
        poly=s[k][1]
        cffs=poly.list()
        dz=dropzeros(cffs)
        dens=listdens(dz)
        pf= sorted(primeFactorsInList(dens))
        if 2 in pf:twoinpf=twoinpf+[index]
        utp=sorted(union([2],pf))
        lastone=lastInList(utp)
        nextpr=next_prime(lastone)
        oldutp=sorted(utp)
        utp=sorted(union(utp,[nextpr]))
        pn= primesToN(index)
        print "--------------------------------------------------------------------------"
        print "index: ",index
        print "pf:"
        print pf
        print "oldutp:"
        print oldutp
        print "(lastone,nextpr):"
        print (lastone,nextpr)
        print "utp:"
        print utp
        print "pn:"
        print pn
        if utp != pn:no=no+[index]
        if utp==pn:yes=yes+[index]
print
print "yes:"
print yes
print
print "no:"
print no
print
print "two in pf:"
print
print twoinpf  

--------------------------------------------------------------------------
index:  2
pf:
[3]
oldutp:
[2, 3]
(lastone,nextpr):
(3, 5)
utp:
[2, 3, 5]
pn:
[2]
--------------------------------------------------------------------------
index:  3
pf:
[]
oldutp:
[2]
(lastone,nextpr):
(2, 3)
utp:
[2, 3]
pn:
[2, 3]
--------------------------------------------------------------------------
index:  5
pf:
[3]
oldutp:
[2, 3]
(lastone,nextpr):
(3, 5)
utp:
[2, 3, 5]
pn:
[2, 3, 5]
--------------------------------------------------------------------------
index:  7
pf:
[3, 5]
oldutp:
[2, 3, 5]
(lastone,nextpr):
(5, 7)
utp:
[2, 3, 5, 7]
pn:
[2, 3, 5, 7]
--------------------------------------------------------------------------
index:  11
pf:
[3, 5, 7]
oldutp:
[2, 3, 5, 7]
(lastone,nextpr):
(7, 11)
utp:
[2, 3, 5, 7, 11]
pn:
[2, 3, 5, 7, 11]
--------------------------------------------------------------------------
index:  13
pf:
[3, 5, 7, 11]
oldutp:
[2, 3, 5, 7, 11]
(lastone,nextpr):
(11, 13)
utp:
[2, 3

--------------------------------------------------------------------------
index:  109
pf:
[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107]
oldutp:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107]
(lastone,nextpr):
(107, 109)
utp:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109]
pn:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109]
--------------------------------------------------------------------------
index:  113
pf:
[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109]
oldutp:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109]
(lastone,nextpr):
(109, 113)
utp:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37,

In [4]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
yes=[]
no=[]
twoinpf=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if is_prime(index)==False:
        
        poly=s[k][1]
        cffs=poly.list()
        dz=dropzeros(cffs)
        dens=listdens(dz)
        pf= sorted(primeFactorsInList(dens))
        if 2 in pf:twoinpf=twoinpf+[index]
        utp=sorted(union([2],pf)) # append no extra prime except 2
        pn= primesToN(index)
        print "--------------------------------------------------------------------------"
        print "index: ",index
        print "pf:"
        print pf
        print "utp:"
        print utp
        print "pn:"
        print pn
        if utp != pn:no=no+[index]
        if utp==pn:yes=yes+[index]
print
print "yes:"
print yes
print
print "no:"
print no
print
print "two in pf:"
print
print twoinpf  

--------------------------------------------------------------------------
index:  -1
pf:
[]
utp:
[2]
pn:
[]
--------------------------------------------------------------------------
index:  0
pf:
[]
utp:
[2]
pn:
[]
--------------------------------------------------------------------------
index:  1
pf:
[]
utp:
[2]
pn:
[]
--------------------------------------------------------------------------
index:  4
pf:
[3, 5]
utp:
[2, 3, 5]
pn:
[2, 3]
--------------------------------------------------------------------------
index:  6
pf:
[3, 5, 7]
utp:
[2, 3, 5, 7]
pn:
[2, 3, 5]
--------------------------------------------------------------------------
index:  8
pf:
[3, 5, 7]
utp:
[2, 3, 5, 7]
pn:
[2, 3, 5, 7]
--------------------------------------------------------------------------
index:  9
pf:
[3, 5, 7]
utp:
[2, 3, 5, 7]
pn:
[2, 3, 5, 7]
--------------------------------------------------------------------------
index:  10
pf:
[3, 5, 7, 11]
utp:
[2, 3, 5, 7, 11]
pn:
[2, 3, 5, 7]
-----------

--------------------------------------------------------------------------
index:  58
pf:
[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59]
utp:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59]
pn:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53]
--------------------------------------------------------------------------
index:  60
pf:
[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61]
utp:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61]
pn:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59]
--------------------------------------------------------------------------
index:  62
pf:
[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61]
utp:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61]
pn:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61]
--------------------------------------------------------------------------
index:  63
pf:
[3, 5, 7, 11, 13, 17,

--------------------------------------------------------------------------
index:  94
pf:
[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89]
utp:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89]
pn:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89]
--------------------------------------------------------------------------
index:  95
pf:
[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89]
utp:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89]
pn:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89]
--------------------------------------------------------------------------
index:  96
pf:
[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
utp:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 

In [ ]:
# no = primes - 1 except that 2 is missing. A006093 URL http://oeis.org/A006093

In [12]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data1=[]
data2=[]
data3=[]
data4=[]
data5=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if lp<pp:
        data1=data1+[index]
        if is_prime(index)==False:
            data2=data2+[index]
            data3= data3+[pp-lp]
    if is_prime(index)==True:
        if (lp<pp)==False:
            data4=data4+[index]
    if (lp<pp)==False:
        if is_prime(index)==False:
            if is_prime(index+1)==False:
                data5=data5+[index]
print data1
print
print data2
print
print data3
print
print data4
print 
print data5

[3, 5, 7, 8, 9, 11, 13, 14, 15, 17, 19, 20, 21, 23, 24, 25, 26, 27, 29, 31, 32, 33, 34, 35, 37, 38, 39, 41, 43, 44, 45, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 59, 61, 62, 63, 64, 65, 67, 68, 69, 71, 73, 74, 75, 76, 77, 79, 80, 81, 83, 84, 85, 86, 87, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 101, 103, 104, 105, 107, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119]

[8, 9, 14, 15, 20, 21, 24, 25, 26, 27, 32, 33, 34, 35, 38, 39, 44, 45, 48, 49, 50, 51, 54, 55, 56, 57, 62, 63, 64, 65, 68, 69, 74, 75, 76, 77, 80, 81, 84, 85, 86, 87, 90, 91, 92, 93, 94, 95, 98, 99, 104, 105, 110, 111, 114, 115, 116, 117, 118, 119]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

[2]

[-1, 0]


In [14]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data1=[]
data2=[]
data3=[]
data4=[]
data5=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if lp<pp:
        data1=data1+[index]
        if is_prime(index)==False:
            data2=data2+[index]
            data3= data3+[pp-lp]
    if is_prime(index)==True:
        if (lp<pp)==False:
            data4=data4+[index]
    if is_prime(index)==False:
            data2=data2+[index]
            data3= data3+[pp-lp]
print data1
print
print data2
print
print data3
print
print data4

[3, 5, 7, 8, 9, 11, 13, 14, 15, 17, 19, 20, 21, 23, 24, 25, 26, 27, 29, 31, 32, 33, 34, 35, 37, 38, 39, 41, 43, 44, 45, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 59, 61, 62, 63, 64, 65, 67, 68, 69, 71, 73, 74, 75, 76, 77, 79, 80, 81, 83, 84, 85, 86, 87, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 101, 103, 104, 105, 107, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119]

[8, 9, 14, 15, 20, 21, 24, 25, 26, 27, 32, 33, 34, 35, 38, 39, 44, 45, 48, 49, 50, 51, 54, 55, 56, 57, 62, 63, 64, 65, 68, 69, 74, 75, 76, 77, 80, 81, 84, 85, 86, 87, 90, 91, 92, 93, 94, 95, 98, 99, 104, 105, 110, 111, 114, 115, 116, 117, 118, 119]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

[2]


In [ ]:
# data2= composite n s.t. n+1 is also composite. A068780. url = http://oeis.org/A068780 
# author Robert G. Wilson v

In [20]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data1=[]
data2=[]
data3=[]
data4=[]
data5=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
    lp=len(pf)
    pp=prime_pi(index)
    if pp-lp<0:data1=data1+[index]
    if pp-lp==0:data2=data2+[index]
    if pp-lp==1:data3=data3+[index]
    if pp-lp==2:data4=data4+[index]
    if pp-lp>2:data5=data5+[index]
print data1;print
print data2;print
print data3;print
print data4;print
print data5
    

[]

[-1, 0, 1, 2, 4, 6, 10, 12, 16, 18, 22, 28, 30, 36, 40, 42, 46, 52, 58, 60, 66, 70, 72, 78, 82, 88, 96, 100, 102, 106, 108, 112]

[8, 9, 14, 15, 20, 21, 24, 25, 26, 27, 32, 33, 34, 35, 38, 39, 44, 45, 48, 49, 50, 51, 54, 55, 56, 57, 62, 63, 64, 65, 68, 69, 74, 75, 76, 77, 80, 81, 84, 85, 86, 87, 90, 91, 92, 93, 94, 95, 98, 99, 104, 105, 110, 111, 114, 115, 116, 117, 118, 119]

[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113]

[]


In [ ]:
# data2 = {-1,0} union n th prime - 1. A006093 author NJA Sloane url http://oeis.org/A006093
# data3 = n s.t. n & n+1 are composite. A068780 author Robt. G. Wilson V.
# data 4 = odd primes

In [26]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    ppi=prime_pi(index)
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= sort(primeFactorsInList(dens))
    lp=len(pf)
    for j in [0..lp-2]:
        p1=pf[j]
        nxt=next_prime(p1)
        p2=pf[j+1]
        if nxt != p2:data=data+[(index,p1,p2,nxt)]
print data 
# data empty says: no missing primes within $\delta_n$ 
        

[]


In [27]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    ppi=prime_pi(index)
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= sort(primeFactorsInList(dens))
    lp=len(pf)
    if lp==0:data=data+[index]
print data 
        

[-1, 0, 1, 3]


In [28]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if index ==2:
        ppi=prime_pi(index)
        poly=s[k][1]
        cffs=poly.list()
        dz=dropzeros(cffs)
        dens=listdens(dz)
        pf= sort(primeFactorsInList(dens))
        print pf

[3]


In [32]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if index >3:
        ppi=prime_pi(index)
        poly=s[k][1]
        cffs=poly.list()
        dz=dropzeros(cffs)
        dens=listdens(dz)
        pf= sort(primeFactorsInList(dens))
        mx=max(pf)
        if mx > index:data=data+[index]
print data

[4, 6, 10, 12, 16, 18, 22, 28, 30, 36, 40, 42, 46, 52, 58, 60, 66, 70, 72, 78, 82, 88, 96, 100, 102, 106, 108, 112]


In [ ]:
# data = odd primes - 1 A006903 author NJA Sloane

In [33]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if index >3:
        ppi=prime_pi(index)
        poly=s[k][1]
        cffs=poly.list()
        dz=dropzeros(cffs)
        dens=listdens(dz)
        pf= sort(primeFactorsInList(dens))
        mx=max(pf)
        if mx == index:data=data+[index]
print data

[]


In [7]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
empty=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if is_prime(index)==True:
        ppi=prime_pi(index)
        poly=s[k][1]
        cffs=poly.list()
        dz=dropzeros(cffs)
        dens=listdens(dz)
        pf= sort(primeFactorsInList(dens))
        if len(pf)==0:empty=empty+[index]
        if len(pf)>0:
            print (index,min(pf),max(pf))
print "empty: "
print empty
        
       

(2, 3, 3)
(5, 3, 3)
(7, 3, 5)
(11, 3, 7)
(13, 3, 11)
(17, 3, 13)
(19, 3, 17)
(23, 3, 19)
(29, 3, 23)
(31, 3, 29)
(37, 3, 31)
(41, 3, 37)
(43, 3, 41)
(47, 3, 43)
(53, 3, 47)
(59, 3, 53)
(61, 3, 59)
(67, 3, 61)
(71, 3, 67)
(73, 3, 71)
(79, 3, 73)
(83, 3, 79)
(89, 3, 83)
(97, 3, 89)
(101, 3, 97)
(103, 3, 101)
(107, 3, 103)
(109, 3, 107)
(113, 3, 109)
empty: 
[3]


In [8]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
empty=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if is_prime(index)==False:
        if is_prime(index+1)==False:
            ppi=prime_pi(index)
            poly=s[k][1]
            cffs=poly.list()
            dz=dropzeros(cffs)
            dens=listdens(dz)
            pf= sort(primeFactorsInList(dens))
            if len(pf)==0:empty=empty+[index]
            if len(pf)>0:
                print (index,min(pf),max(pf))
print "empty: "
print empty
        
       

(8, 3, 7)
(9, 3, 7)
(14, 3, 13)
(15, 3, 13)
(20, 3, 19)
(21, 3, 19)
(24, 3, 23)
(25, 3, 23)
(26, 3, 23)
(27, 3, 23)
(32, 3, 31)
(33, 3, 31)
(34, 3, 31)
(35, 3, 31)
(38, 3, 37)
(39, 3, 37)
(44, 3, 43)
(45, 3, 43)
(48, 3, 47)
(49, 3, 47)
(50, 3, 47)
(51, 3, 47)
(54, 3, 53)
(55, 3, 53)
(56, 3, 53)
(57, 3, 53)
(62, 3, 61)
(63, 3, 61)
(64, 3, 61)
(65, 3, 61)
(68, 3, 67)
(69, 3, 67)
(74, 3, 73)
(75, 3, 73)
(76, 3, 73)
(77, 3, 73)
(80, 3, 79)
(81, 3, 79)
(84, 3, 83)
(85, 3, 83)
(86, 3, 83)
(87, 3, 83)
(90, 3, 89)
(91, 3, 89)
(92, 3, 89)
(93, 3, 89)
(94, 3, 89)
(95, 3, 89)
(98, 3, 97)
(99, 3, 97)
(104, 3, 103)
(105, 3, 103)
(110, 3, 109)
(111, 3, 109)
(114, 3, 113)
(115, 3, 113)
(116, 3, 113)
(117, 3, 113)
(118, 3, 113)
(119, 3, 113)
empty: 
[-1, 0]


In [10]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
empty=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if is_prime(index)==False:
        if is_prime(index+1)==True:
            ppi=prime_pi(index)
            poly=s[k][1]
            cffs=poly.list()
            dz=dropzeros(cffs)
            dens=listdens(dz)
            pf= sort(primeFactorsInList(dens))
            if len(pf)==0:empty=empty+[index]
            if len(pf)>0:
                print (index,min(pf),max(pf))
print "empty: "
print empty

(4, 3, 5)
(6, 3, 7)
(10, 3, 11)
(12, 3, 13)
(16, 3, 17)
(18, 3, 19)
(22, 3, 23)
(28, 3, 29)
(30, 3, 31)
(36, 3, 37)
(40, 3, 41)
(42, 3, 43)
(46, 3, 47)
(52, 3, 53)
(58, 3, 59)
(60, 3, 61)
(66, 3, 67)
(70, 3, 71)
(72, 3, 73)
(78, 3, 79)
(82, 3, 83)
(88, 3, 89)
(96, 3, 97)
(100, 3, 101)
(102, 3, 103)
(106, 3, 107)
(108, 3, 109)
(112, 3, 113)
empty: 
[1]


In [11]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
empty=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if index==2:
        ppi=prime_pi(index)
        poly=s[k][1]
        cffs=poly.list()
        dz=dropzeros(cffs)
        dens=listdens(dz)
        pf= sort(primeFactorsInList(dens))
        if len(pf)==0:empty=empty+[index]
        if len(pf)>0:
            print (index,min(pf),max(pf))
print "empty: "
print empty

(2, 3, 3)
empty: 
[]
